# Modify this variable to adapt to your storage in you're cloud

In [1]:
# project location in Google drive
project_location_drive = '/content/drive/MyDrive/Fac/Master/CV'
project_location_local = '/content/'

#
data_location_local = "/content/data/media/nas/01_Datasets/CT/LITS/"

# data_location_local = "data_skip/"

In [2]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.9/413.9 kB 32.9 MB/s eta 0:00:00


In [3]:
import wandb

In [4]:
! wandb login

wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tpoux (tumor-3D) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
! ln -s /content/drive/MyDrive/Fac/Master/optuna /content/optuna

In [ ]:
# copy all the folder so the data leave on the same host has the GPU used
! cp -rv /content/drive/MyDrive/Fac/Master/CV/* /content/

'/content/drive/MyDrive/Fac/Master/CV/checkpoints' -> '/content/checkpoints'
'/content/drive/MyDrive/Fac/Master/CV/checkpoints/run_continued_last.pth' -> '/content/checkpoints/run_continued_last.pth'
'/content/drive/MyDrive/Fac/Master/CV/checkpoints/run_continued_best.pth' -> '/content/checkpoints/run_continued_best.pth'
'/content/drive/MyDrive/Fac/Master/CV/checkpoints/checkpoint_last.pth' -> '/content/checkpoints/checkpoint_last.pth'
'/content/drive/MyDrive/Fac/Master/CV/checkpoints/checkpoint_best.pth' -> '/content/checkpoints/checkpoint_best.pth'
'/content/drive/MyDrive/Fac/Master/CV/data' -> '/content/data'
'/content/drive/MyDrive/Fac/Master/CV/data/Training_Batch2.zip' -> '/content/data/Training_Batch2.zip'
'/content/drive/MyDrive/Fac/Master/CV/data/Training_Batch1.zip' -> '/content/data/Training_Batch1.zip'


In [ ]:
# unzip dataset in //
!unzip /content/data/Training_Batch1.zip -d /content/data/ & \
 unzip /content/data/Training_Batch2.zip -d /content/data/ & \
 wait

In [ ]:
! rm -r  /content/data/Training_Batch1.zip
! rm -r  /content/data/Training_Batch2.zip

In [ ]:
import torch
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import random
from torch.utils.data import DataLoader, Subset, WeightedRandomSampler
from dataset.LiTSPatchDataset import *
from utility.dice_score import dice_loss_fg, dice_eval
from utility.log_image import log_images_wandb
from tqdm import tqdm
from torch import sigmoid
from torch.amp import autocast, GradScaler
from UNet.unet_model import UNet
import os, re
import gc
import optuna

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)
if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))

In [ ]:
def build_pairs(*batch_dirs):
    pairs = []
    for d in batch_dirs:
        files = os.listdir(d)

        seg_files = [f for f in files if "segmentation" in f]
        for seg in sorted(seg_files):
            m = re.search(r"segmentation-(\d+)", seg)
            if not m:
                continue
            idx = m.group(1)


            vol_candidates = [
                f"volume-{idx}.nii",
            ]
            vol = next((v for v in vol_candidates if v in files), None)
            if vol is None:
                raise FileNotFoundError(f"Volume manquant pour {seg} dans {d}")

            pairs.append((os.path.join(d, vol), os.path.join(d, seg)))
    return pairs


pairs = build_pairs(os.path.join(data_location_local ,"Training Batch 1/"), os.path.join(data_location_local ,"Training Batch 2/"))
print("Nb paires:", len(pairs))
print(pairs[0])


X_paths = [p[0] for p in pairs]
y_paths = [p[1] for p in pairs]

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

In [ ]:
def split_patients(patient_ids, train=0.8, val=0.15, test=0.05, seed=42):
    assert abs(train + val + test - 1.0) < 1e-9
    ids = list(patient_ids)
    random.Random(seed).shuffle(ids)

    n = len(ids)
    n_train = int(train * n)
    n_val = int(val * n)

    train_ids = ids[:n_train]
    val_ids   = ids[n_train:n_train+n_val]
    test_ids  = ids[n_train+n_val:]
    return train_ids, val_ids, test_ids

def make_paths(batch_dirs, patient_ids):
    X, Y = [], []
    for pid in patient_ids:
        for d in batch_dirs:
            vol = os.path.join(d, f"volume-{pid}.nii")
            seg = os.path.join(d, f"segmentation-{pid}.nii")
            if os.path.exists(vol) and os.path.exists(seg):
                X.append(vol); Y.append(seg)
                break
    return X, Y

max_in_paths = 0
for d in [os.path.join(data_location_local ,"Training Batch 1/"), os.path.join(data_location_local ,"Training Batch 2/")]:
        files = os.listdir(d)

        seg_files = [f for f in files if "volume" in f]
        for seg in sorted(seg_files):
            m = re.search(r"volume-(\d+)", seg)
            if not m:
                continue
            if int(m[1]) > max_in_paths:
                 max_in_paths = int(m[1])
            else:
                 continue

patient_ids = np.arange(max_in_paths+1)

train_ids, val_ids, test_ids = split_patients(patient_ids)



X_train_paths, y_train_paths  = make_paths([os.path.join(data_location_local ,"Training Batch 1/"), os.path.join(data_location_local ,"Training Batch 2/")], train_ids)
X_val_paths, y_val_paths= make_paths([os.path.join(data_location_local ,"Training Batch 1/"), os.path.join(data_location_local ,"Training Batch 2/")], val_ids)
X_test_paths, y_test_paths= make_paths([os.path.join(data_location_local ,"Training Batch 1/"), os.path.join(data_location_local ,"Training Batch 2/")], test_ids)
train_dataset = LiTSPatchDataset(X_train_paths,y_train_paths)
val_dataset = LiTSPatchDataset(X_val_paths,y_val_paths)
test_dataset = LiTSPatchDataset(X_test_paths,y_test_paths)



In [ ]:
def flush_memory():
    # force le Garbage Collector de Python
    gc.collect()
    # vide le cache CUDA
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()


In [ ]:
def unet_train(
    model,
    train_dataset,
    val_dataset,
    device,
    epochs: int = 10,
    batch_size: int = 1,
    l_rate: float = 1e-4,
    save_checkpoints: bool = True,
    weight_decay: float = 1e-5,
    patience: int = 7,
    min_delta: float = 0.005,
    run_name: str = "run",
    early_stop: bool = True,
    trial=None
):


    use_amp = (device.type == "cuda")
    scaler = GradScaler("cuda", enabled=use_amp)

    loader_args = dict(batch_size=batch_size, num_workers=4, pin_memory=True)




    train_loader = DataLoader(
        train_dataset,
        batch_size=loader_args["batch_size"],
        num_workers=loader_args["num_workers"],
        pin_memory=loader_args["pin_memory"],
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=loader_args["batch_size"],
        num_workers=loader_args["num_workers"],
        pin_memory=loader_args["pin_memory"],
        shuffle=False,
        drop_last=False
    )

    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=l_rate,
        betas=(0.9, 0.999),
        weight_decay=weight_decay
    )

    pos_weight = torch.tensor([25.0], device=device)
    bce_loss = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)


    val_mri_idx = [0,1,2,3,4,5]


    viz_subset = Subset(val_dataset, val_mri_idx)

    losses_train, losses_val, dices_val = [], [], []

    best_dice = -1.0
    epochs_no_improve = 0


    for epoch in range(epochs):
        model.train()
        cum_loss = 0.0
        nb_batches = 0

        for x_batch, y_batch in tqdm(train_loader, desc=f"Train epoch {epoch}", leave=False):
            x = x_batch.to(device)
            y = y_batch.to(device)

            if y.dim() == 3:
                y = y.unsqueeze(1)
            y = y.float()

            optimizer.zero_grad(set_to_none=True)

            with autocast(enabled=use_amp, device_type="cuda" if torch.cuda.is_available() else "cpu"):
                logits = model(x)
                loss = dice_loss_fg(logits, y) + bce_loss(logits, y)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            cum_loss += float(loss.item())
            nb_batches += 1
            wandb.log({"loss_train": float(loss.item())})


        loss_epoch = cum_loss / nb_batches if nb_batches > 0 else float("nan")

        wandb.log({"epoch": epoch, "loss_train": loss_epoch})

        model.eval()
        cum_loss_val = 0.0
        nb_val_batches = 0
        dice_sum = 0.0
        n_sum = 0

        with torch.no_grad():
            for x_batch, y_batch in tqdm(val_loader, desc=f"Val epoch {epoch}", leave=False):
                x = x_batch.to(device)
                y = y_batch.to(device)

                if y.dim() == 3:
                    y = y.unsqueeze(1)
                y = y.float()

                with autocast(enabled=use_amp, device_type="cuda" if torch.cuda.is_available() else "cpu"):
                    logits = model(x)
                    batch_loss_val = dice_loss_fg(logits, y) + bce_loss(logits, y)

                dice_batch, n_valid = dice_eval(logits, y)
                dice_sum += float(dice_batch) * int(n_valid)
                n_sum += int(n_valid)

                cum_loss_val += float(batch_loss_val.item())
                nb_val_batches += 1

        dice_epoch = (dice_sum / n_sum) if n_sum > 0 else float("nan")
        loss_val_epoch = (cum_loss_val / nb_val_batches) if nb_val_batches > 0 else float("nan")

        losses_train.append(loss_epoch)
        losses_val.append(loss_val_epoch)
        dices_val.append(dice_epoch)

        wandb.log({"epoch": epoch, "loss_val": loss_val_epoch, "dice_val": dice_epoch})



        if epoch % 2 == 0:
            log_images_wandb(model,
                            val_dataset,
                            device,
                            indices=val_mri_idx,
                            epoch=epoch,
                            threshold=0.5,
                            alpha=0.45)


        if save_checkpoints:
            ckpt = {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "best_dice": best_dice,
                "loss_train": losses_train,
                "loss_val": losses_val,
                "dice_val": dices_val,
            }
            torch.save(ckpt, f"{run_name}_last.pth")

        if trial is not None:
            trial.report(dice_epoch, epoch)

            # Check if the trial should be pruned (stopped early)
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

        if early_stop:

            if not np.isnan(dice_epoch) and dice_epoch > best_dice + min_delta:
                best_dice = dice_epoch
                epochs_no_improve = 0

                if save_checkpoints:
                    ckpt["best_dice"] = best_dice
                    torch.save(ckpt, f"{run_name}_best.pth")
            else:
                epochs_no_improve += 1

            if epochs_no_improve >= patience:
                print(f"Early stopping triggered at epoch {epoch}!")
                break

    artifact = wandb.Artifact('Unet', type='model')
    if os.path.exists(f"{run_name}_best.pth"):
        artifact.add_file(f"{run_name}_best.pth")
    else :
        artifact.add_file(f"{run_name}_last.pth")
    wandb.log_artifact(artifact)





    return losses_train, losses_val, dices_val


In [ ]:
def objective(trial):
    if device.type == "cuda":
        flush_memory() # clear memory before starting a new trial, we can't afford to crash because of out-of-memory ;)

    # hyper parameters to optimize
    dropout = trial.suggest_float("dropout", 0.0, 0.3, step=0.1)
    l_rate = trial.suggest_float("l_rate", 1e-5, 1e-3, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    bilinear = trial.suggest_categorical("bilinear", [True, False])




    hyperparameters = {
        "dropout": dropout,
        "l_rate": l_rate,
        "weight_decay": weight_decay,
        "bilinear": bilinear,
        "max_epochs": 120,
        "patience": 10,
        "min_delta": 0.001,
        "batch_size": 32,
        "early_stop": True
    }


    model = UNet(n_channels=1, n_classes=1, bilinear=hyperparameters["bilinear"], dropout=hyperparameters["dropout"]).to(device)

    # parameters logging
    project_name = "tumor-3D"
    group_name = "optuna-study-1"

    wandb.init(project=project_name, config=hyperparameters, name=f"trial-{trial.number}", group=group_name)

    print(f"Starting trial {trial.number} with parameters: {[dropout, l_rate, weight_decay, bilinear]}")
    try :
        _, _, dices_val = unet_train(
            model,
            train_dataset,
            val_dataset,
            device,
            epochs=hyperparameters["max_epochs"],
            batch_size=hyperparameters["batch_size"],
            l_rate=hyperparameters["l_rate"],
            weight_decay=hyperparameters["weight_decay"],
            patience=hyperparameters["patience"],
            min_delta=hyperparameters["min_delta"],
            early_stop=hyperparameters["early_stop"],
            trial=trial,
            run_name=f"trial-{trial.number}"
        )

    except optuna.exceptions.TrialPruned:
        wandb.finish()
        print(f"Trial {trial.number} pruned.")
        return 0.0

    except Exception as e:
        print(f"Trial {trial.number} failed with exception: {e}")
        wandb.finish()
        return 0.0



    best_dice = np.max(dices_val) if len(dices_val) > 0 else 0.0

    wandb.finish()

    print(f"Trial {trial.number} completed with best dice: {best_dice}")

    return best_dice

In [ ]:
def create_conf_study():
    """
    Create a configuration for Optuna.
    """

    storage = f"sqlite:///optuna/optuna_study__{"optuna-study-1"}.db"
    sampler = optuna.samplers.TPESampler()
    pruner = optuna.pruners.MedianPruner(n_startup_trials=3, n_warmup_steps=5, interval_steps=3)

    return storage, sampler, pruner

In [ ]:
# create study db if needed

storage, sampler, pruner = create_conf_study()

study = optuna.create_study( direction="maximize", study_name="optuna-study-1", storage=storage, sampler=sampler, pruner=pruner, load_if_exists=True)



# launch optimization
study.optimize(objective, n_trials=100)